In [ ]:
!apt-get install sumo sumo-tools sumo-doc
!pip install traci numpy torch matplotlib pandas


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  binfmt-support fastjar fonts-roboto-unhinted jarwrapper javascript-common libcoin80c
  libcollada-dom2.5-dp0 libfox-1.6-0 libgdal30 libglu1-mesa libjs-openlayers liblua5.2-0
  libopenscenegraph161 libopenthreads21 libpoppler-glib8 libproj22 proj-bin
Suggested packages:
  apache2 | lighttpd | httpd libopenal0a libsimage-dev libbullet2
The following NEW packages will be installed:
  binfmt-support fastjar fonts-roboto-unhinted jarwrapper javascript-common libcoin80c
  libcollada-dom2.5-dp0 libfox-1.6-0 libgdal30 libglu1-mesa libjs-openlayers liblua5.2-0
  libopenscenegraph161 libopenthreads21 libpoppler-glib8 libproj22 proj-bin sumo sumo-doc
  sumo-tools
0 upgraded, 20 newly installed, 0 to remove and 30 not upgraded.
Need to get 52.8 MB of archives.
After this operation, 270 MB of additional disk space will be used.
Get:1 http://archive.

In [ ]:
import csv
import sumolib

def extract_and_save_network_data(network_file, output_file="network_data.csv"):
    # Load the network using sumolib
    net = sumolib.net.readNet(network_file)

    # Extract and save edge information
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Edge ID", "From Node", "To Node", "Length (m)", "Speed Limit (m/s)", "Number of Lanes"])

        for edge in net.getEdges():
            # Fix for Excel issue by adding a single quote to Edge IDs
            edge_id = f"'{edge.getID()}"

            writer.writerow([
                edge_id,
                edge.getFromNode().getID(),
                edge.getToNode().getID(),
                edge.getLength(),
                edge.getSpeed(),
                len(edge.getLanes())
            ])

    print(f"Data successfully saved to {output_file}")

# Provide your network file (.net.xml)
network_file = "/content/updated.net.xml"
extract_and_save_network_data(network_file)


Data successfully saved to network_data.csv


In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import random

# Load CSV files
data_path = "/content/all_edge_data.csv"
network_path = "/content/network_data.csv"

data = pd.read_csv(data_path)  # Real-time traffic data
network_data = pd.read_csv(network_path)  # Road network data

# Inspect data
print("🚦 Traffic Data Sample:")
print(data.head())

print("\n🛣️ Network Data Sample:")
print(network_data.head())


🚦 Traffic Data Sample:
   step   edge_id  avg_speed  vehicle_count  occupancy  traffic_flow
0   100      '-E0   0.000000              0   0.000000      0.000000
1   100      '-E1   0.000000              0   0.000000      0.000000
2   100  '-E1.703   0.000000              0   0.000000      0.000000
3   100     '-E10  20.978846              1   0.467942     20.978846
4   100     '-E11   0.000000              0   0.000000      0.000000

🛣️ Network Data Sample:
    Edge ID From Node To Node  Length (m)  Speed Limit (m/s)  Number of Lanes
0      '-E0        J2      J1      988.39               19.0                3
1      '-E1        J3     J18      708.14               19.0                3
2  '-E1.703       J18      J2      581.44               19.0                3
3     '-E10       J13     J12      558.83               19.0                2
4     '-E11       J14     J13      704.63               19.0                3


In [ ]:
import pandas as pd
import networkx as nx

def create_directed_graph(network_file):
    df = pd.read_csv(network_file)

    # Calculate travel time
    df["time"] = df["Length (m)"] / df["Speed Limit (m/s)"]
    df["distance"] = df["Length (m)"]  # For consistency

    # Create directed graph
    G = nx.DiGraph()

    # Add edges with weights and edge ID
    for _, row in df.iterrows():
        G.add_edge(
            row["From Node"],
            row["To Node"],
            edge_id=row["Edge ID"],         # ✅ Add edge ID
            distance=row["distance"],
            time=row["time"]
        )

    return G

# Path to your network CSV
network_file_path = '/content/network_data.csv'
graph = create_directed_graph(network_file_path)

print("✅ Graph created with:")
print(f"Nodes: {graph.number_of_nodes()}, Edges: {graph.number_of_edges()}")


✅ Graph created with:
Nodes: 34, Edges: 84


In [ ]:
# Convert graph to adjacency matrix
adj_matrix = nx.adjacency_matrix(graph)

# Convert to DataFrame for readability
adj_df = pd.DataFrame(adj_matrix.todense(), index=graph.nodes(), columns=graph.nodes())

# Display the adjacency matrix
adj_df.head()


,J2,J1,J3,J18,J13,J12,J14,J4,J16,J19,...,J34,J5,J11,J40,J45,J42,J43,J7,J44,J8
J2,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
J1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
J3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J18,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J13,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [ ]:
import networkx as nx

def find_all_routes_with_edges(graph, start_node, end_node, max_length=None):
    try:
        # Find all node-based paths
        all_routes = list(nx.all_simple_paths(graph, source=start_node, target=end_node, cutoff=max_length))
        edge_routes = []

        # Convert node paths to edge paths
        for path in all_routes:
            edge_path = []
            for u, v in zip(path[:-1], path[1:]):
                if (u, v) in graph.edges:
                    edge_id = graph.edges[u, v].get('edge_id')
                    edge_path.append(str(edge_id))
            edge_routes.append(edge_path)

        print(f"✅ Total Routes from {start_node} to {end_node}: {len(edge_routes)}")
        return edge_routes

    except nx.NetworkXNoPath:
        print(f"⚠️ No path found between {start_node} and {end_node}")
        return []

# Example usage
start_node = 'J1'  # Replace with your actual start node
end_node = 'J10'    # Replace with your actual end node
routes_with_edges = find_all_routes_with_edges(graph, start_node, end_node)

# ✅ Display edge-based routes without quotes or extra characters
for i, route in enumerate(routes_with_edges, 1):
    formatted_route = " -> ".join(str(edge).strip("'") for edge in route)
    print(f"Route {i}: {formatted_route}")


✅ Total Routes from J1 to J10: 144
Route 1: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> -E18
Route 2: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> E25 -> E26 -> E27 -> -E35 -> E36 -> -E9.302 -> -E8
Route 3: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> E25 -> E26 -> E27 -> E28 -> E41 -> -E9 -> -E9.302 -> -E8
Route 4: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> E25 -> E26 -> E34 -> E35 -> E28 -> E41 -> -E9 -> -E9.302 -> -E8
Route 5: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> E25 -> E26 -> E34 -> E36 -> -E9.302 -> -E8
Route 6: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> E25 -> E33 -> -E8
Route 7: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> E37 -> -E29 -> -E28 -> -E27 -> -E26 -> E33 -> -E8
Route 8: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> E37 -> -E29 -> -E28 -> -E27 -> E34 -> E36 -> -E9.302 -> -E8
Route 9: E0 -> E1 -> E1.596 -> E2 -> -E32 -> -E21 -> -E19 -> E37 -> -E29 -> -E28 -> -E35 -> -E34 -> -E26 -> E33 -> -E8
Route 10: 

In [ ]:
import csv

input_file = '/content/all_edge_data.csv'
output_file = 'all_edge_data_cleaned.csv'

with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    header = next(reader)  # Copy header
    writer.writerow(header)

    for row in reader:
        if not row or len(row) < 6:
            continue  # Skip empty or malformed rows

        edge_id = row[1].lstrip("'")  # Remove single quote
        if edge_id.startswith(":"):
            continue  # Skip internal edges

        try:
            avg_speed = float(row[2])
            vehicle_count = int(float(row[3]))  # Convert vehicle count safely
        except ValueError:
            continue  # Skip invalid numeric data

        if vehicle_count == 0 or avg_speed == 0:
            continue  # Skip zero-traffic data

        row[1] = edge_id  # Update cleaned edge_id back in row
        writer.writerow(row)


In [ ]:
import pandas as pd

# Load data
data = pd.read_csv("/content/all_edge_data_cleaned.csv")
print(data.head(100))


    step edge_id  avg_speed  vehicle_count  occupancy  traffic_flow
0    100    -E10  20.978846              1   0.467942     20.978846
1    100    -E12  18.802583              1   0.289989     18.802583
2    100    -E13  18.020909              3   1.000383     54.062726
3    100    -E15  15.464651              2   0.556756     30.929301
4    100    -E16  20.160676              2   0.340483     40.321351
..   ...     ...        ...            ...        ...           ...
95   300     E11  22.388820              1   0.231726     22.388820
96   300     E12  16.609491              3   0.668554     49.828473
97   300     E18  19.413409              1   0.235329     19.413409
98   300     E19  18.613622              2   0.839543     37.227245
99   300      E2  11.934880              4   1.031776     47.739518

[100 rows x 6 columns]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tqdm import tqdm

# Load the traffic data
traffic_data = pd.read_csv('/content/all_edge_data_cleaned.csv')

# Dictionary to hold predicted traffic_flow and travel times
predicted_traffic = {}
predicted_travel_time = {}

# Parameters
sequence_length = 10
epochs = 5  # For demo; increase for better training
batch_size = 16

# Loop over all unique edges
edge_ids = traffic_data['edge_id'].unique()

for edge_id in tqdm(edge_ids, desc="Training LSTM per edge"):
    # Extract and sort data for the current edge
    edge_data = traffic_data[traffic_data['edge_id'] == edge_id].sort_values(by='step')

    # Skip if not enough data
    if len(edge_data) <= sequence_length:
        continue

    # Normalize traffic flow
    scaler = MinMaxScaler()
    edge_data['traffic_flow_scaled'] = scaler.fit_transform(edge_data[['traffic_flow']])

    # Create sequences
    X, y = [], []
    for i in range(len(edge_data) - sequence_length):
        X.append(edge_data['traffic_flow_scaled'].iloc[i:i+sequence_length].values)
        y.append(edge_data['traffic_flow_scaled'].iloc[i+sequence_length])

    X = np.array(X).reshape(-1, sequence_length, 1)
    y = np.array(y)

    # Define and train LSTM model
    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)),
        LSTM(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)

    # Predict the next time step
    last_seq = edge_data['traffic_flow_scaled'].values[-sequence_length:].reshape(1, sequence_length, 1)
    pred_scaled = model.predict(last_seq)[0][0]
    pred_flow = scaler.inverse_transform([[pred_scaled]])[0][0]

    # Save prediction
    predicted_traffic[edge_id] = pred_flow

    # Get latest avg_speed for that edge (or use default if NaN)
    latest_speed = edge_data['avg_speed'].values[-1]
    edge_length = edge_data['avg_speed'].values[-1] * (edge_data['step'].iloc[-1] / 1000)  # just an example

    # If speed is too low or zero, assign minimum speed to avoid division by zero
    if latest_speed <= 0:
        latest_speed = 2.0  # m/s

    predicted_travel_time[edge_id] = edge_length / latest_speed

print("\n✅ Done predicting traffic flow and travel times for all edges!")


Training LSTM per edge:   0%|          | 0/79 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step


Training LSTM per edge:   1%|▏         | 1/79 [00:06<07:50,  6.03s/it]/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step


Training LSTM per edge:   3%|▎         | 2/79 [00:11<07:32,  5.88s/it]/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step


Training LSTM per edge:  95%|█████████▍| 75/79 [07:11<00:22,  5.66s/it]/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step


Training LSTM per edge: 100%|██████████| 79/79 [07:16<00:00,  5.53s/it]


✅ Done predicting traffic flow and travel times for all edges!


In [ ]:
import networkx as nx

# Assuming your graph `G` is already created using network_data.csv
# and `predicted_travel_time` dict is ready from LSTM models

# Update edge weights
for u, v, data in graph.edges(data=True):
    edge_id = data.get("edge_id")

    if edge_id in predicted_travel_time:
        # Set predicted time as new weight
        data["predicted_time"] = predicted_travel_time[edge_id]
    else:
        # If no prediction is available, fall back to default time estimate
        default_speed = data.get("speed_limit", 10)  # m/s
        length = data.get("length", 100)  # m
        data["predicted_time"] = length / default_speed

print("✅ Graph updated with predicted travel times.")


✅ Graph updated with predicted travel times.


In [ ]:
import pandas as pd
import random

# Define fixed source and destination
source_node = "J1"       # Replace with your actual source node
destination_node = "J10" # Replace with your actual destination node

# States and actions
states = list(graph.nodes)
actions = {node: list(graph.successors(node)) for node in states}

# Initialize Q-table with zeros
Q_table = pd.DataFrame(0, index=states, columns=states)

# Hyperparameters
gamma = 0.8  # Discount factor
alpha = 0.5  # Learning rate
episodes = 500

# Q-Learning Training Loop
for _ in range(episodes):
    current_state = source_node

    while current_state != destination_node and actions[current_state]:
        next_state = random.choice(actions[current_state])

        # Get predicted travel time from edge data
        edge_data = graph.get_edge_data(current_state, next_state)
        travel_time = edge_data.get("predicted_time", 100)  # fallback if not set

        # Assign reward
        if next_state == destination_node:
            reward = 100  # Strong positive reward for reaching destination
        else:
            reward = -travel_time  # Negative reward to minimize travel time

        # Q-learning update
        max_future_q = Q_table.loc[next_state].max()
        current_q = Q_table.loc[current_state, next_state]

        new_q = (1 - alpha) * current_q + alpha * (reward + gamma * max_future_q)
        Q_table.loc[current_state, next_state] = new_q

        # Move to next state
        current_state = next_state

print("✅ Q-Learning training complete.")


<ipython-input-73-f56ff409f46d>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-7.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Q_table.loc[current_state, next_state] = new_q
<ipython-input-73-f56ff409f46d>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-7.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Q_table.loc[current_state, next_state] = new_q
<ipython-input-73-f56ff409f46d>:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-7.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Q_table.loc[current_state, next_state] = new_q
<ipython-input-73-f56ff409f46d>:42: FutureWarning: Setting an item of incompati

✅ Q-Learning training complete.


In [ ]:
def find_best_path_rl(start, end, max_steps=100):
    path_nodes = [start]
    path_edges = []
    total_time = 0.0
    total_distance = 0.0
    current = start
    visited = set()

    for _ in range(max_steps):
        if current == end:
            return path_nodes, path_edges, total_time, total_distance

        visited.add(current)

        if Q_table.loc[current].sum() == 0:
            print(f"⚠️ No learned Q-values for node {current}. Stopping.")
            return None, None, None, None

        next_node = Q_table.loc[current].idxmax()

        if next_node in visited:
            print(f"⚠️ Loop detected at node {next_node}. Stopping.")
            return None, None, None, None

        edge_info = graph.get_edge_data(current, next_node)

        if not edge_info or "edge_id" not in edge_info:
            print(f"❌ Edge from {current} to {next_node} missing 'edge_id'. Got: {edge_info}")
            return None, None, None, None

        # Clean edge ID (remove any extra quotes or spaces)
        raw_edge_id = edge_info["edge_id"]
        edge_id = raw_edge_id.strip().replace("'", "")

        travel_time = edge_info.get("predicted_time", 0.0)
        distance = edge_info.get("distance", 0.0)

        path_edges.append(edge_id)
        total_time += travel_time
        total_distance += distance

        path_nodes.append(next_node)
        current = next_node

    print("⛔ Max steps reached. Possibly infinite loop.")
    return None, None, None, None

# 🧪 Example usage
nodes_path, edges_path, total_time, total_distance = find_best_path_rl("J1", "J10")

if nodes_path and edges_path:
    print("🧭 Best path (nodes): " + " -> ".join(nodes_path))
    print("🛣️ Best path (edges): " + " -> ".join(edges_path))
    print(f"⏱️ Total predicted time: {total_time:.2f} seconds")
    print(f"📏 Total distance: {total_distance:.2f} meters")


🧭 Best path (nodes): J1 -> J2 -> J10
🛣️ Best path (edges): E0 -> E6
⏱️ Total predicted time: 20.00 seconds
📏 Total distance: 1977.95 meters
